# My First Blog Post

This is my very first blog on GitHub Pages! I wanted to make it fun, but also a place where I can reflect on what I’ve already done in **Computer Science Principles** so far.

## Jupyter Notebooks

One of the first tools I learned was **Jupyter Notebooks**.  
It felt like a mix between coding and note-taking, which made it really useful for experimenting with Python. I could write code, run it, and instantly see the output underneath. For example, I remember writing a simple `for` loop that printed jokes one line at a time—like a knock-knock joke that didn’t need an audience.

## Managing Files

Another important skill was **file management**.  
I worked with creating, renaming, and organizing files so that projects didn’t turn into a giant mess. It wasn’t just about dragging and dropping—it was about learning how the **command line** can control files, and how GitHub keeps track of all the changes. It’s like having a time machine for code.

## GitHub Pages

The coolest thing so far has been setting up **GitHub Pages**.  
I learned how to turn a simple repository into a live website (just like this blog). I had to use front matter (the section above this post) to control the layout, title, and description. It made me realize how coding isn’t only about solving problems—it can also be creative, like designing your own corner of the internet.

## Debugging (and Jokes)

Of course, no programming class is complete without a little debugging.  
Sometimes my code didn’t run, and I had to figure out why. But every bug also gave me a laugh. Here’s one of my favorite computer jokes so far:

> Why do programmers prefer dark mode?  
> Because the light attracts bugs. 🐛

---

This blog is just the start. I’ll keep adding posts as I learn more—whether it’s Python tricks, web design, or just more bad programming jokes.


90+ → A
80–89 → B
70–79 → C
60–69 → D
<60 → F


---

## 🧠 Backend API

We built a REST API using **Flask** to serve the model:

POST /api/grade/predict
GET /api/grade/predict (JWT-protected)


### POST `/predict`

- Accepts JSON with an `inputs` list of 11 values (each between 1 and 5)
- Returns predicted percent and letter grade

### GET `/predict` *(JWT Protected)*

- Returns the **saved grade prediction** from the user's profile

This ensures that users can retrieve their grade later without needing to re-enter their inputs.

---

## 🔐 Authentication

We protect the GET endpoint using JWT-based authentication. Only logged-in users can retrieve their saved predictions, which are stored in the database under their user profile.

---

## 🧩 Frontend Integration

We tackled a few issues to make the frontend fully functional:

### ✅ [Issue #28](https://github.com/luojonah/palomarhealth_frontend/issues/28)
We ensured that after a user submits their self-assessment, the **predicted grade is saved to their profile**.

### ✅ [Issue #32](https://github.com/luojonah/palomarhealth_frontend/issues/32)
We added logic so that when users **log back in**, their previously saved prediction is displayed.

---

## 🌐 Tech Stack

- **Frontend**: HTML / Bootstrap (Palomar Health UI)
- **Backend**: Flask, Flask-RESTful, JWT Authorization
- **ML**: Scikit-learn (Linear Regression)
- **Data**: Custom CSV dataset of self-reported academic habits and grades

---

# 💻 Raw Backend Code (API + Model)


## grade_api and Predict API Resource

In [ ]:
from flask import Blueprint, request, jsonify, g
from flask_restful import Api, Resource
from api.jwt_authorize import token_required
from model.user import User
from model.grade_model import GradePredictionModel

grade_api = Blueprint('grade_api', __name__, url_prefix='/api/grade')
api = Api(grade_api)

model_instance = GradePredictionModel()

class Predict(Resource):
    def post(self):
        data = request.get_json()

        if not data or 'inputs' not in data:
            return {"error": "Missing 'inputs' field in JSON payload. Expected a list of 11 numbers (1-5)."}, 400

        user_input = data['inputs']

        if len(user_input) != 11:
            return {"error": f"Expected 11 input values, received {len(user_input)}."}, 400

        try:
            user_input = [int(val) for val in user_input]
        except ValueError:
            return {"error": "All input values must be numbers between 1 and 5."}, 400

        if not all(1 <= val <= 5 for val in user_input):
            return {"error": "Input values should be between 1 and 5."}, 400

        percent, letter = model_instance.predict(user_input)

        return jsonify({
            'predicted_percent': percent,
            'predicted_grade': letter
        })

    @token_required()
    def get(self):
        user = g.current_user
        return jsonify(user.grade_data)

api.add_resource(Predict, '/predict')


## 🔹 GradePredictionModel Class

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

class GradePredictionModel:
    def __init__(self):
        data = pd.read_csv("datasets/ap_predict_data.csv")
        data = data.dropna(subset=['Grade'])
        data.columns = data.columns.str.strip()

        grade_map = {'A': 90, 'B': 80, 'C': 70, 'D': 60, 'F': 50}
        data['GradePercent'] = data['Grade'].map(grade_map)

        self.features = ['Attendance','Work Habits','Behavior','Timeliness','Advocacy','Tech Growth',
                         'Tech Sense','Tech Talk','Communication and Collaboration','Leadership','Integrity']

        X = data[self.features]
        y = data['GradePercent']

        self.model = LinearRegression()
        self.model.fit(X, y)

    def predict(self, user_input):
        if len(user_input) != len(self.features):
            raise ValueError(f"Expected {len(self.features)} inputs, got {len(user_input)}")

        binarized = [0 if x <= 3 else 1 for x in user_input]
        percent = self.model.predict([binarized])[0]
        percent = max(0, min(100, percent))

        if percent >= 90:
            letter = 'A'
        elif percent >= 80:
            letter = 'B'
        elif percent >= 70:
            letter = 'C'
        elif percent >= 60:
            letter = 'D'
        else:
            letter = 'F'

        return round(percent, 2), letter
